In [1]:
# Google Colaboratoryを使用する場合は、下記のコメントアウトを外し、実行してください

In [2]:
# # 下記セルを実行すると、authorization codeの入力を求められます。
# # 出力されたリンク先をクリックし、Googleアカウントにログインし、
# # authorization codeをコピーし、貼り付けをおこなってください。
# import os
# from google.colab import drive
# drive.mount('/content/drive')

In [3]:
# working_dir = 'MLSys_100Knocks' #　※※自分で作成したフォルダパスが異なる場合こちらを変更してください。※※
# path = f'/content/drive/MyDrive/{working_dir}/本章/7章'
# os.chdir(path)

# ７章 機械学習モデルを構築する１０本ノック



### ノック６１：フォルダ生成をして機械学習用データを読み込もう

In [4]:
import os
data_dir = "data"
input_dir = os.path.join(data_dir, "0_input")
output_dir = os.path.join(data_dir, "1_output")
os.makedirs(input_dir, exist_ok=True)
os.makedirs(output_dir, exist_ok=True)


##### ＊＊必ずデータの配置をおこなってください。＊＊

In [5]:
import pandas as pd
ml_data_file = "ml_base_data.csv"
ml_data = pd.read_csv(os.path.join(input_dir, ml_data_file))
ml_data.head(3)

,store_name,y_weekday,y_weekend,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,...,order_time_14,order_time_15,order_time_16,order_time_17,order_time_18,order_time_19,order_time_20,order_time_21,delta_avg,year_month
0,あきる野店,1.0,0.0,1147,945,202,841,306,844,303,...,101,95,107,106,100,108,109,96,34.110053,201904
1,さいたま南店,1.0,1.0,1504,1217,287,1105,399,1104,400,...,143,142,137,130,113,140,132,155,35.337716,201904
2,さいたま緑店,1.0,1.0,1028,847,181,756,272,756,272,...,95,102,82,90,93,95,95,84,34.291617,201904


### ノック６２：カテゴリカル変数の対応をしよう

In [6]:
category_data = pd.get_dummies(ml_data["store_name"], prefix="store", prefix_sep="_")
display(category_data.head(3))

,store_あきる野店,store_さいたま南店,store_さいたま緑店,store_さいたま西店,store_つくば店,store_三浦店,store_三鷹店,store_上尾店,store_上野店,store_世田谷店,...,store_駒込店,store_高円寺店,store_高島平店,store_高崎店,store_高座店,store_高津店,store_高田馬場店,store_鴻巣店,store_鶴見店,store_麻生店
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
del category_data["store_麻生店"]
del ml_data["year_month"]
del ml_data["store_name"]
ml_data = pd.concat([ml_data, category_data], axis=1)
ml_data.columns

Index(['y_weekday', 'y_weekend', 'order', 'order_fin', 'order_cancel',
       'order_delivery', 'order_takeout', 'order_weekday', 'order_weekend',
       'order_time_11',
       ...
       'store_駒沢店', 'store_駒込店', 'store_高円寺店', 'store_高島平店', 'store_高崎店',
       'store_高座店', 'store_高津店', 'store_高田馬場店', 'store_鴻巣店', 'store_鶴見店'],
      dtype='object', length=215)

### ノック６３：学習データとテストデータを分割しよう

In [8]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(ml_data, test_size=0.3, random_state=0)
print(f'Train:{len(train_data)}件/ Test:{len(test_data)}')
print(f'Weekday Train0:{len(train_data.loc[train_data["y_weekday"]==0])}')
print(f'Weekday Train1:{len(train_data.loc[train_data["y_weekday"]==1])}')
print(f'Weekday Test0:{len(test_data.loc[test_data["y_weekday"]==0])}件')
print(f'Weekday Test1:{len(test_data.loc[test_data["y_weekday"]==1])}件')
print(f'Weekend Train0:{len(train_data.loc[train_data["y_weekend"]==0])}')
print(f'Weekend Train1:{len(train_data.loc[train_data["y_weekend"]==1])}')
print(f'Weekend Test0:{len(test_data.loc[test_data["y_weekend"]==0])}件')
print(f'Weekend Test1:{len(test_data.loc[test_data["y_weekend"]==1])}件')


Train:1501件/ Test:644
Weekday Train0:685
Weekday Train1:816
Weekday Test0:290件
Weekday Test1:354件
Weekend Train0:708
Weekend Train1:793
Weekend Test0:295件
Weekend Test1:349件


### ノック６４：１つのモデルを構築しよう

In [9]:
X_cols = list(train_data.columns)
X_cols.remove("y_weekday")
X_cols.remove("y_weekend")
target_y = "y_weekday"
y_train = train_data[target_y]
X_train = train_data[X_cols]
y_test = test_data[target_y]
X_test = test_data[X_cols]
display(y_train.head(3))
display(X_train.head(3))

1137    1.0
971     0.0
1983    1.0
Name: y_weekday, dtype: float64

,order,order_fin,order_cancel,order_delivery,order_takeout,order_weekday,order_weekend,order_time_11,order_time_12,order_time_13,...,store_駒沢店,store_駒込店,store_高円寺店,store_高島平店,store_高崎店,store_高座店,store_高津店,store_高田馬場店,store_鴻巣店,store_鶴見店
1137,977,809,168,724,253,685,292,102,88,84,...,0,0,0,0,0,0,0,0,0,0
971,1099,904,195,816,283,779,320,99,102,101,...,0,0,0,0,0,0,0,1,0,0
1983,966,794,172,724,242,671,295,80,95,87,...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(random_state=0)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=0)

### ノック６５：評価を実施してみよう

In [11]:
y_pred_train = model.predict(X_train)
y_pred_test = model.predict(X_test)
y_pred_test

array([0., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
       0., 0., 1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 1.,
       1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
       0., 0., 1., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1.,
       1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 0., 0., 1.,
       0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0.,
       1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 1.,
       0., 1., 0., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 1., 1., 0., 0.,
       0., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 1., 1.,
       0., 1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 0., 0.,
       1., 0., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0.,
       1., 0., 1., 1., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0., 1., 1., 0.,
       0., 1., 1., 0., 0.

In [12]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score, confusion_matrix
acc_train = accuracy_score(y_train, y_pred_train)
acc_test = accuracy_score(y_test, y_pred_test)
f1_train = f1_score(y_train, y_pred_train)
f1_test = f1_score(y_test, y_pred_test)
recall_train = recall_score(y_train, y_pred_train)
recall_test = recall_score(y_test, y_pred_test)
precision_train = precision_score(y_train, y_pred_train)
precision_test = precision_score(y_test, y_pred_test)
print(f'【正解率】Train:{round(acc_train,2)} Test:{round(acc_test, 2)}')
print(f'【F値】Train: {round(f1_train,2)} Test:{round(f1_test,2)}')
print(f'【再現率】Train:{round(recall_train,2)} Test:{round(recall_test,2)}')
print(f'【適合率】Train:{round(precision_train,2)} Test:{round(precision_test,2)}')

【正解率】Train:1.0 Test:0.82
【F値】Train: 1.0 Test:0.84
【再現率】Train:1.0 Test:0.82
【適合率】Train:1.0 Test:0.86


In [13]:
print(confusion_matrix(y_train, y_pred_train))
print(confusion_matrix(y_test, y_pred_test))

[[685   0]
 [  0 816]]
[[241  49]
 [ 64 290]]


In [14]:
tn_train, fp_train, fn_train, tp_train = confusion_matrix(y_train, y_pred_train).ravel()
tn_test, fp_test, fn_test, tp_test = confusion_matrix(y_test, y_pred_test).ravel()
print(f'【混同行列】Train:{tn_train}, {fp_train}, {fn_train}, {tp_train}')
print(f'【混同行列】Test:{tn_test}, {fp_test}, {fn_test}, {tp_test}')

【混同行列】Train:685, 0, 0, 816
【混同行列】Test:241, 49, 64, 290


In [15]:
score_train = pd.DataFrame({"DataCategory":["train"], "acc":[acc_train], "f1":[f1_train], "recall":[recall_train], "precision":[precision_train], "tp":[tp_train], "fn":[fn_train], "fp":[fp_train], "tn":[tn_train]})
score_test = pd.DataFrame({"DataCategory":["test"], "acc":[acc_test], "f1":[f1_test], "recall":[recall_test], "precision":[precision_test], "tp":[tp_test], "fn":[fn_test], "fp":[fp_test], "tn":[tn_test]})
score = pd.concat([score_train, score_test], ignore_index=True)
score


,DataCategory,acc,f1,recall,precision,tp,fn,fp,tn
0,train,1.000000,1.000000,1.000000,1.000000,816,0,0,685
1,test,0.824534,0.836941,0.819209,0.855457,290,64,49,241


### ノック６６：モデルの重要度を確認してみよう

In [16]:
importance = pd.DataFrame({"cols":X_train.columns, "importance":model.feature_importances_})
importance = importance.sort_values("importance", ascending=False)
importance.head(10)

,cols,importance
5,order_weekday,0.369241
6,order_weekend,0.346013
18,delta_avg,0.027430
2,order_cancel,0.026031
12,order_time_16,0.024161
8,order_time_12,0.023256
3,order_delivery,0.020037
17,order_time_21,0.018936
10,order_time_14,0.017932
11,order_time_15,0.017163


### ノック６７：モデル構築から評価までを関数化しよう

### ノック６８：モデルファイルや評価結果を出力しよう

### ノック６９：アルゴリズムを拡張して多角的な評価を実施しよう

### ノック７０：平日/休日モデルを一度で回せるようにしよう